##### Татур А.А.
##### v.0.1 02.11.2022
##### Данные (взяты из ДЗ_12).
##### Модели: GaussianNB, LinearRegression. Выполнен подбор параметров GridSearchCV

##### Цели:

1. Использовать полученные знания в части обучения моделей для обучения и подбора параметров в задачах из предыдущих ДЗ
2. Использовать: GaussianNB, LinearRegression, PolynomialFeatures.
3. При обучении использовать make_pipeline, GridSearchCV

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures


from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

In [4]:
# Читаем файл. Сразу убираем колонку id, формируем фрэйм с фичами и серию с целевым признаком
df = pd.read_csv('MFOcredit.csv',
                 encoding = 'cp1251',
                 sep=';')
df=df.drop('id',axis=1)

X=df.drop(['delinq60plus'], axis=1)
X.age=X.age.astype(np.float16)
y=df['delinq60plus']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.7, random_state=42)

#### Обрабатываем пропуски. Кодируем бинарные колонки в цифровые вручную. Походу приводим типы к float

##### 1. Кодируем целевую переменную

In [6]:
#Кодируем целевую колонку. Да=1, Нет=0
y_train.loc[(y_train=='Да')] = 1
y_train.loc[(y_train=='Нет')] = 0

y_test.loc[(y_test=='Да')] = 1
y_test.loc[(y_test=='Нет')] = 0

#Приводим к типу int
y_train=y_train.astype(np.float16)
y_test=y_test.astype(np.float16)

##### 2. Обрабатываем даты. 

In [7]:
#Меняем типы на дату в тренировочных колонках
X_train.date_start=X_train.date_start.astype('datetime64[ns]')
X_train.date_end=X_train.date_end.astype('datetime64[ns]')

#Меняем типы на дату в тестовых колонках
X_test.date_start=X_test.date_start.astype('datetime64[ns]')
X_test.date_end=X_test.date_end.astype('datetime64[ns]')

In [8]:
# Этот ужас переводит дату в формат "Год.Дни недели с начала года", чтобы их можно было запихивать в модельку.
# Такое усложнение понадобилось, т.к. если просто вызвать функцию dayofyear для числа, если прошло меньше чем 100 дней
# c начала года, то происходит следующее, допустим 30 января 2010 года превратится в 2010.30, т.е. 300й день. Поэтому циклы
# проверяют и если дней меньше ста допихивают ноль спереди.

for i in X_train.index:
    if X_train.loc[i, 'date_start'].dayofyear<100:
        X_train.loc[i, 'y.days_start'] = str(X_train.loc[i, 'date_start'].year)+'.0'+\
        str(X_train.loc[i, 'date_start'].dayofyear)
    else:
        X_train.loc[i, 'y.days_start'] = str(X_train.loc[i, 'date_start'].year)+'.'+\
        str(X_train.loc[i, 'date_start'].dayofyear)
X_train['y.days_start']=X_train['y.days_start'].astype(float)

for i in X_train.index:
    if X_train.loc[i, 'date_end'].dayofyear<100:
        X_train.loc[i, 'y.days_end'] = str(X_train.loc[i, 'date_end'].year)+'.0'+\
        str(X_train.loc[i, 'date_end'].dayofyear)
    else:
        X_train.loc[i, 'y.days_end'] = str(X_train.loc[i, 'date_end'].year)+'.'+\
        str(X_train.loc[i, 'date_end'].dayofyear)
X_train['y.days_end']=X_train['y.days_end'].astype(float)

for i in X_test.index:
    if X_test.loc[i, 'date_start'].dayofyear<100:
        X_test.loc[i, 'y.days_start'] = str(X_test.loc[i, 'date_start'].year)+'.0'+\
        str(X_test.loc[i, 'date_start'].dayofyear)
    else:
        X_test.loc[i, 'y.days_start'] = str(X_test.loc[i, 'date_start'].year)+'.'+\
        str(X_test.loc[i, 'date_start'].dayofyear)
X_test['y.days_start']=X_test['y.days_start'].astype(float)

for i in X_test.index:
    if X_test.loc[i, 'date_end'].dayofyear<100:
        X_test.loc[i, 'y.days_end'] = str(X_test.loc[i, 'date_end'].year)+'.0'+\
        str(X_test.loc[i, 'date_end'].dayofyear)
    else:
        X_test.loc[i, 'y.days_end'] = str(X_test.loc[i, 'date_end'].year)+'.'+\
        str(X_test.loc[i, 'date_end'].dayofyear)
X_test['y.days_end']=X_test['y.days_end'].astype(float)

In [9]:
#Вытаскиваем разность между датами. Получаем длину контракта.
X_train['credit_duration']=(X_train['date_end']-X_train['date_start']).dt.days
X_test['credit_duration']=(X_test['date_end']-X_test['date_start']).dt.days

In [10]:
#Выкидываем исходные колонки
X_train=X_train.drop(['date_start', 'date_end'], axis=1)
X_test=X_test.drop(['date_start', 'date_end'], axis=1)

##### 3. Кодируем пол

In [11]:
#Обработка колонки пола. 0 - Женский, 1 - Мужской

X_train.loc[(X_train.gender == 'Женский'), 'gender'] = '0'
X_train.loc[(X_train.gender == 'Мужской'), 'gender'] = '1'

X_test.loc[(X_test.gender == 'Женский'), 'gender'] = '0'
X_test.loc[(X_test.gender == 'Мужской'), 'gender'] = '1'

#Приводим к типу int
X_train.gender=X_train.gender.astype(np.float16)
X_test.gender=X_test.gender.astype(np.float16)

##### 4. Обрабатываем пропуски, кодируем колонку auto

In [12]:
#Заменяем пропуски в колонке "auto" на np.nan.
#X_train.loc[(X_train.auto == 'Пропуск поля'), 'auto'] = np.nan
#X_test.loc[(X_test.auto == 'Пропуск поля'), 'auto'] = np.nan

#Кодируем все в бинарные значения. 0 - нет авто. 1 - есть авто
#X_train.loc[(X_train.auto == 'Нет'), 'auto'] = '0'
#X_train.loc[(X_train.auto == 'Да'), 'auto'] = '1'

#X_test.loc[(X_test.auto == 'Нет'), 'auto'] = '0'
#X_test.loc[(X_test.auto == 'Да'), 'auto'] = '1'

#Конвертируем тип в int
#X_test['auto']=X_test['auto'].astype(np.float16)
#X_train['auto']=X_train['auto'].astype(np.float16)

#####  5. Обарабываем пропуски в колонке housing

In [13]:
#Заменяем пропуски в колонке 'housing' на np.nan
X_train.loc[(X_train.housing=='Пропуск поля'), 'housing'] = np.nan
X_test.loc[(X_test.housing=='Пропуск поля'), 'housing'] = np.nan

##### 6. Обарабываем пропуски в колонке marstatus

In [14]:
#Заменяем пропуски в 'marstatus' на np.nan
X_train.loc[(X_train.marstatus == 'Пропуск поля'), 'marstatus'] =np.nan
X_test.loc[(X_test.marstatus == 'Пропуск поля'), 'marstatus'] = np.nan

#####  7. Кодируем колонку regclient

In [15]:
#Кодируем regclien в бинарные значения. Да = 1, нет=0. Приводим к типу int16

X_train.loc[(X_train.regclient == 'Нет'), 'regclient'] = '0'
X_train.loc[(X_train.regclient == 'Да'), 'regclient'] = '1'

X_test.loc[(X_test.regclient == 'Нет'), 'regclient'] = '0'
X_test.loc[(X_test.regclient == 'Да'), 'regclient'] = '1'

#Приводим к типу int
X_train.regclient=X_train.regclient.astype(np.float16)
X_test.regclient=X_test.regclient.astype(np.float16)

#####  8. Обарабываем пропуски в колонке jobtype и кодируем

In [16]:
#Заменяем пропуски в колонке "jobtype" на np.nan
X_train.loc[(X_train.jobtype == 'Пропуск поля'), 'jobtype'] = np.nan
X_test.loc[(X_test.jobtype == 'Пропуск поля'), 'jobtype'] = np.nan

#Кодируем все в бинарные значения. 0 - неоффициальное. 1 - оффициальное
X_train.loc[(X_train.jobtype == 'Неофициальное'), 'jobtype'] = '0'
X_train.loc[(X_train.jobtype == 'Официальное'), 'jobtype'] = '1'

X_test.loc[(X_test.jobtype == 'Неофициальное'), 'jobtype'] = '0'
X_test.loc[(X_test.jobtype == 'Официальное'), 'jobtype'] = '1'

#Конвертируем тип в int
X_test['jobtype']=X_test['jobtype'].astype(np.float16)
X_train['jobtype']=X_train['jobtype'].astype(np.float16)

#####  9. Обарабываем пропуски в колонке credis и кодируем

In [17]:
#Заменяем пропуски в колонке "credits" на np.nan
X_train.loc[(X_train.credits == 'Пропуск поля'), 'credits'] = np.nan
X_test.loc[(X_test.credits == 'Пропуск поля'), 'credits'] = np.nan

#Кодируем все в бинарные значения. 0 - нет. 1 - да
X_train.loc[(X_train.credits == 'Нет'), 'credits'] = '0'
X_train.loc[(X_train.credits == 'Да'), 'credits'] = '1'

X_test.loc[(X_test.credits == 'Нет'), 'credits'] = '0'
X_test.loc[(X_test.credits == 'Да'), 'credits'] = '1'

#Конвертируем тип в int
X_test['credits']=X_test['credits'].astype(np.float16)
X_train['credits']=X_train['credits'].astype(np.float16)

#####  9. Обарабываем пропуски в колонке children и кодируем

In [18]:
#Заменяем пропуски в колонке "children" на np.nan.
X_train.loc[(X_train.children == 'Пропуск поля'), 'children'] = np.nan
X_test.loc[(X_test.children == 'Пропуск поля'), 'children'] = np.nan

#Кодируем все в бинарные значения. 0 - нет. 1 - да
X_train.loc[(X_train.children == 'Нет'), 'children'] = '0'
X_train.loc[(X_train.children == 'Да'), 'children'] = '1'

X_test.loc[(X_test.children == 'Нет'), 'children'] = '0'
X_test.loc[(X_test.children == 'Да'), 'children'] = '1'

#Конвертируем тип в int
X_test['children']=X_test['children'].astype(np.float16)
X_train['children']=X_train['children'].astype(np.float16)

#####  10. Применяем Dummies кодирование к вещественным колонкам с более чем 2-мя признаками.

In [19]:
X_train_dummie = pd.concat([X_train,
                           pd.get_dummies(X_train['auto'], prefix='auto'),
                           pd.get_dummies(X_train['housing'], prefix='housing'),
                           pd.get_dummies(X_train['marstatus'], prefix='marstatus'),
                           pd.get_dummies(X_train['jobtype'], prefix='jobtype'),
                           pd.get_dummies(X_train['credits'], prefix='credits'),
                           pd.get_dummies(X_train['children'], prefix='children'),
                           pd.get_dummies(X_train['region'], prefix='region')], axis=1)
X_test_dummie = pd.concat([X_test,
                           pd.get_dummies(X_test['auto'], prefix='auto'),
                           pd.get_dummies(X_test['housing'], prefix='housing'),
                           pd.get_dummies(X_test['marstatus'], prefix='marstatus'),
                           pd.get_dummies(X_test['jobtype'], prefix='jobtype'),
                           pd.get_dummies(X_test['credits'], prefix='credits'),
                           pd.get_dummies(X_test['children'], prefix='children'),
                           pd.get_dummies(X_test['region'], prefix='region')], axis=1)

#####  11. Удаляем вещественные колонки

In [20]:
X_train_dummie = X_train_dummie.drop(['auto','housing','marstatus','jobtype','credits','children','region'],axis=1)
X_test_dummie = X_test_dummie.drop(['auto','housing','marstatus','jobtype','credits','children','region'],axis=1)

#####  12. Обучаем модели
##### 12.1 LinearRegression

In [27]:
linear = make_pipeline (GridSearchCV(LinearRegression(),
                                     param_grid = { 'n_jobs': range(-10,10)},
                                     cv=2,
                                     refit=True,
                                     verbose=1))
linear.fit (X_train_dummie, y_train)
y_lin_predict = linear.predict(X_test_dummie)
roc_auc_score(y_test, y_lin_predict)

Fitting 2 folds for each of 20 candidates, totalling 40 fits


0.837650546970274

##### 12.2 GaussianNaiveBayes

In [31]:
gnb = make_pipeline (GridSearchCV(GaussianNB(),
                                     param_grid = { 'var_smoothing': np.logspace(0,-9, num=100)},
                                     cv=2,
                                     refit=True,
                                     verbose=1))
gnb.fit (X_train_dummie, y_train)
y_gnb_predict_proba = gnb.predict_proba(X_test_dummie)[:,1]
roc_auc_score(y_test, y_gnb_predict_proba)

Fitting 2 folds for each of 100 candidates, totalling 200 fits


0.8014362478370249

##### 12.3 LinearRegression + PolynomialFeatures

In [33]:
linear_poly = make_pipeline (PolynomialFeatures(),
                        GridSearchCV(LinearRegression(),
                                     param_grid = { 'n_jobs': range(-10,10)},
                                     cv=2,
                                     refit=True,
                                     verbose=1))
linear_poly.fit (X_train_dummie, y_train)
y_lin_predict_poly = linear_poly.predict(X_test_dummie)
roc_auc_score(y_test, y_lin_predict_poly)

Fitting 2 folds for each of 20 candidates, totalling 40 fits


0.8436566085349557

##### 12.4 GaussianNiveBayes + PolynomialFeatures

In [35]:
gnb_poly = make_pipeline (PolynomialFeatures(),
                          GridSearchCV(GaussianNB(),
                                     param_grid = { 'var_smoothing': np.logspace(0,-9, num=100)},
                                     cv=2,
                                     refit=True,
                                     verbose=1))
gnb_poly.fit (X_train_dummie, y_train)
y_gnb_predict_proba_poly = gnb_poly.predict_proba(X_test_dummie)[:,1]
roc_auc_score(y_test, y_gnb_predict_proba_poly)

Fitting 2 folds for each of 100 candidates, totalling 200 fits


0.7831718775071334